# IMPORTS

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime, date
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

# PLOTTING PARAMETERS

In [2]:
plt.style.use("bmh")

plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
    )

plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
    )

# PATHS

In [3]:
path_data = Path('./input')

#  FUNCTIONS TO BE SET IN MODULE

In [4]:
def prepare_seasonal_plot(df):
    
    X = df.copy()

    # days within a week
    X["day"] = X.date.dt.dayofweek  # the x-axis (freq)
    X["week"] = X.date.dt.week  # the seasonal period (period)
    X['dayname'] = X.date.dt.day_name()
    
    # days within a year
    X["dayofyear"] = X.date.dt.dayofyear
    X["year"] = X.date.dt.year

    return X

def seasonal_plot(X, y, period, freq, ax=None):
    
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

# DATA

## Store_sales

In [5]:
store_sales = pd.read_csv(
    path_data / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={'store_nbr': 'category',
           'family': 'category',
           'sales': 'float32',
          },
    parse_dates=['date'],
    infer_datetime_format=True,
    )

pd.to_datetime(store_sales['date'])

store_sales = prepare_seasonal_plot(store_sales)

/tmp/ipykernel_404992/857166004.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  X["week"] = X.date.dt.week  # the seasonal period (period)


fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(11, 6))
seasonal_plot(store_sales.query("family == 'SEAFOOD'"), y="sales", period="week", freq="day", ax=ax0)
seasonal_plot(store_sales.query("family == 'SEAFOOD'"), y="sales", period="year", freq="dayofyear", ax=ax1);

## Stores

In [6]:
stores = pd.read_csv(
    path_data / 'stores.csv',
    dtype={'store_nbr': 'category',
           'city': 'category',
           'state': 'category',
           'type': 'category',
           'cluster': 'category'
           },
    )
# type already use as column name
stores.rename(columns={'type': 'store_type'}, inplace=True)
stores.head()

,store_nbr,city,state,store_type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


## Test data

In [7]:
df_test = pd.read_csv(
    path_data / 'test.csv',
    dtype={'store_nbr': 'category',
           'family': 'category',
           'onpromotion': 'uint32',
           },
    parse_dates=['date'],
    infer_datetime_format=True,
    )

pd.to_datetime(df_test['date'])
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


## Holidays_events

#### DATA DESCRIPTION

Holidays and Events, with metadata

NOTE: 

Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge. Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

ADDITIONAL NOTES:

Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.

A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.



In [8]:
holidays_events = pd.read_csv(
    path_data / "holidays_events.csv",
    dtype={'type': 'category',
          'locale': 'category',
          'locale_name': 'category',
          'description': 'category',
          'transferred': 'bool',#
          },
    parse_dates=['date'],
    infer_datetime_format=True,
    )

pd.to_datetime(holidays_events['date'])
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [9]:
holidays_events.type.unique()

['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event']
Categories (6, object): ['Additional', 'Bridge', 'Event', 'Holiday', 'Transfer', 'Work Day']

- 'Holiday'are holiday, 
- 'Work Day' are extra, unusual work days, meant to compensate for Bridge days 

We create a 'status' column to specify if the given day is a work day or a holiday 

In [10]:
holidays_events['status'] = np.nan

In [11]:
holidays_events.loc[holidays_events.type=='Holiday', 'status'] = 'holiday'
holidays_events.loc[holidays_events.type=='Work Day', 'status'] = 'work'

NOTE: for a few entries (days), two different events occur on the same day

ALSO: same holiday listed twice under 'bridge' and 'additional'

To facilitate the process, 
- we rename some of the events
- more importantly, we rename 'Bridge' as 'Additional' and remove (3) duplicate entries

DEAL WITH BRIDGE/ADDITIONAL DUPLICATES

In [12]:
# cosmetic renaming
holidays_events["description"] = holidays_events["description"].str.replace("Puente ", "")
# rename 'Bridge' as 'Additional'
holidays_events.loc[holidays_events.type=='Bridge', 'type'] = 'Additional'
# select 'Additional' rows
additional = holidays_events.loc[holidays_events.type=='Additional']
print(len(additional))
# drop 'Bridge' vs 'Additional' duplicates
new_additional=additional.drop_duplicates(subset=['date', 'locale'], keep='first', \
                                          inplace=False, ignore_index=False)
print(len(new_additional))

print(len(holidays_events))
# remove 'additional' from the original df
holidays_events.drop(holidays_events.loc[holidays_events['type']=='Additional'].index, inplace=True)
# Appends new_additional (that is, with duplicates removed) to the original df
holidays_events = pd.concat([holidays_events, new_additional], axis=0)
print(len(holidays_events))

56
53
350
347


- 'Additional' are holidays

In [13]:
holidays_events.loc[holidays_events.type=='Additional', 'status'] = 'holiday'

- 'Transfer' are holidays (initially scheduled another day, transferred to a 'Transfer' day)

Since the initial information on the holiday is provided, there is no need to further manipulated the data to keep track of that

In [14]:
holidays_events.loc[holidays_events.type=='Transfer', 'status'] = 'holiday'

- 'Transferred' are work days (initially scheduled as holiday, transferred to a 'Transfer' day)

In [15]:
holidays_events.loc[holidays_events.transferred=='True', 'status'] = 'work'

- Events are various, but always National

In [16]:
event_mask = holidays_events.type == 'Event'
event = holidays_events[event_mask]
print(len(event))
event

56


,date,type,locale,locale_name,description,transferred,status
55,2013-05-12,Event,National,Ecuador,Dia de la Madre,False,NaN
103,2014-05-11,Event,National,Ecuador,Dia de la Madre,False,NaN
106,2014-06-12,Event,National,Ecuador,Inauguracion Mundial de futbol Brasil,False,NaN
107,2014-06-15,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Suiza,False,NaN
108,2014-06-20,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Honduras,False,NaN
113,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False,NaN
114,2014-06-28,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,NaN
115,2014-06-29,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,NaN
116,2014-06-30,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,NaN
117,2014-07-01,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,NaN


Event provides additional information

The earthquake and following days are marked as Terremoto Manabi+XX

In a general case, their impact would have to be dealt with, for instance not including them in the training sample

NB: Assessing their impact can only be done by including multiple years in the training sample, which is out of the scope of the kaggle competition since the training range is et from 2017-05-01 to 2017-08-15

Some other days may have direct (Dia de la Madre, black Friday, cyber Monday) or indirect impact (Mundial de futbol Brasil) on (categorical) sales 

Events should be considered as work day (unless they are a holiday for some other reason)

In [17]:
print(len(holidays_events))
# Add boolean column to keep track of event
holidays_events['Terremoto'] = np.where((holidays_events['type']=='Event') & (holidays_events['description'].str.contains('Terremoto')), True, False)
holidays_events['Futbol'] = np.where((holidays_events['type']=='Event') & (holidays_events['description'].str.contains('futbol')), True, False)
holidays_events['CyberMonday'] = np.where((holidays_events['type']=='Event') & (holidays_events['description'].str.contains('Monday')), True, False)
holidays_events['BlackFriday'] = np.where((holidays_events['type']=='Event') & (holidays_events['description'].str.contains('Friday')), True, False)
holidays_events['Madre'] = np.where((holidays_events['type']=='Event') & (holidays_events['description'].str.contains('Madre')), True, False)
# Remove 'Event' events (the info is preserved via the above commands)
holidays_events = holidays_events.loc[holidays_events.type != 'Event']
print(len(holidays_events))
holidays_events

347
291


,date,type,locale,locale_name,description,transferred,status,Terremoto,Futbol,CyberMonday,BlackFriday,Madre
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,holiday,False,False,False,False,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,holiday,False,False,False,False,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,holiday,False,False,False,False,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,holiday,False,False,False,False,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,holiday,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
343,2017-12-21,Additional,National,Ecuador,Navidad-4,False,holiday,False,False,False,False,False
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,holiday,False,False,False,False,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,holiday,False,False,False,False,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,holiday,False,False,False,False,False


#### Holidays can be national, regional, or local

- The location of the store has to be taken into account to verify if a given event is indeed a holiday for a given store
- This leads to complicated issues, in particular because of duplicated dates

- The way to go is:
1) split holidays_events in national/regional/local
2) remove national holidays from regional/local holidays (no need to account for them twice)   
NB: some (descriptive) information is lost in this process
2) merge them separately with store_sales+stores, taking into account 'date' AND 'city'/'state'

##### Merge Store_sales and stores

In [18]:
print(len(store_sales))
store_sales2 = store_sales.merge(stores, on='store_nbr', how='left')
store_sales2
len(store_sales2)

3000888


3000888

In [19]:
national = holidays_events.loc[holidays_events.locale == 'National'].copy()
list_national = national['date'].to_list()
# No duplicated dates
# No NaNs in status

In [20]:
# Merge store_sales2 with national --> full
nat_merged = store_sales2.merge(national, left_on=['date'], right_on=['date'], how='left') 
print(len(nat_merged))

3000888


In [21]:
regional = holidays_events.loc[holidays_events.locale == 'Regional'].copy()
regional.head()
# No duplicated dates
# No NaNs in status

,date,type,locale,locale_name,description,transferred,status,Terremoto,Futbol,CyberMonday,BlackFriday,Madre
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,holiday,False,False,False,False,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False,holiday,False,False,False,False,False
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False,holiday,False,False,False,False,False
24,2012-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False,holiday,False,False,False,False,False
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,holiday,False,False,False,False,False


In [22]:
# No overlap between national holidays and regional holidays
print(len(regional))
print(len(regional.loc[~regional.date.isin(list_national)]))
list_regional = regional['date'].to_list()

24
24


In [23]:
# Merge store_sales2 with regional
reg_merged = store_sales2.merge(regional, left_on=['date', 'state'], right_on=['date', 'locale_name'], how='left', suffixes=[None, '_reg']) 
print(len(reg_merged))

3000888


In [24]:
# check that event in the given region are indeed holidays
print(reg_merged.loc[reg_merged.locale_name.isin(reg_merged.state)]['status'].unique())
# check that non-matching region/state are not holidays (NaNs will be filled in later)
print(reg_merged.loc[~reg_merged.locale_name.isin(reg_merged.state)]['status'].unique())

['holiday']
[nan]


In [25]:
local = holidays_events.loc[holidays_events.locale == 'Local'].copy()
local.loc[local.duplicated(subset=['date','locale_name'], keep=False)]
# There are duplicated dates, but they correspond to different cities, except for 2016-07-24 (fundacion de Guayaquil), 
# which is duplicated for the same city
# No NaNs in status

,date,type,locale,locale_name,description,transferred,status,Terremoto,Futbol,CyberMonday,BlackFriday,Madre
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False,holiday,False,False,False,False,False
264,2016-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,holiday,False,False,False,False,False


In [26]:
# remove this duplicated date
print(len(local))
local.drop_duplicates(subset=['date', 'locale_name'], keep='first', inplace=True, ignore_index=False)
print(len(local))

152
151


In [27]:
# Remove national+regional holidays that were already accounted for from local 
list_natreg = list_national + list_regional
print(len(local))
local_small = local.loc[~local.date.isin(list_natreg)]
print(len(local_small))
local_small

151
131


,date,type,locale,locale_name,description,transferred,status,Terremoto,Futbol,CyberMonday,BlackFriday,Madre
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,holiday,False,False,False,False,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,holiday,False,False,False,False,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,holiday,False,False,False,False,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,holiday,False,False,False,False,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,holiday,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
200,2015-12-05,Additional,Local,Quito,Fundacion de Quito-1,False,holiday,False,False,False,False,False
286,2016-12-05,Additional,Local,Quito,Fundacion de Quito-1,False,holiday,False,False,False,False,False
321,2017-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,holiday,False,False,False,False,False
322,2017-07-25,Additional,Local,Guayaquil,Fundacion de Guayaquil,False,holiday,False,False,False,False,False


In [28]:
# Merge store_sales2 with local_small
loc_merged = store_sales2.merge(local_small, left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left', suffixes =[None, '_loc']) 
print(len(loc_merged))

3000888


In [29]:
# check that event in the given region are indeed holidays
print(loc_merged.loc[loc_merged.locale_name.isin(loc_merged.city)]['status'].unique())

['holiday']


In [30]:
# check that non-matching region/city are not holidays (NaNs will be filled in later)
print(loc_merged.loc[~loc_merged.locale_name.isin(loc_merged.city)]['status'].unique())

[nan]


In [31]:
# concatenate the 3 big dfs (nat_merged, reg_merged, loc_merged)
full=pd.concat([nat_merged, reg_merged, loc_merged], axis=0)

In [32]:
full=pd.concat([nat_merged, reg_merged, loc_merged], axis=0).sort_values(by='status').drop_duplicates(subset=['date', 'store_nbr', 'family'], keep='first', \
                                                                             inplace=False, ignore_index=False)


In [33]:
len(full)

3000888

In [34]:
len(full.loc[~full.status.isna()])

172755

In [35]:
len(full.loc[~full.locale.isna()])

172755

In [36]:
full.loc[full.duplicated(subset=['date', 'store_nbr', 'family'])==True]

,date,store_nbr,family,sales,onpromotion,day,week,dayname,dayofyear,year,...,locale,locale_name,description,transferred,status,Terremoto,Futbol,CyberMonday,BlackFriday,Madre


In [37]:
full.locale.unique()

['National', 'Local', 'Regional', NaN]
Categories (3, object): ['Local', 'National', 'Regional']

In [38]:
# fill NaNs in status with workday 
full.loc[full.status.isna(), 'status'] = 'work'
# fill NaNs in various events with False 
full.loc[full.Terremoto.isna(), 'Terremoto'] = False
full.loc[full.Futbol.isna(), 'Futbol'] = False
full.loc[full.CyberMonday.isna(), 'CyberMonday'] = False
full.loc[full.BlackFriday.isna(), 'BlackFriday'] = False
full.loc[full.Madre.isna(), 'Madre'] = False

In [39]:
# Select relevant columns only
# Basically we get rid of 'type' and 'description'
clean = full[['date', 'store_nbr', 'family', 'sales', 'onpromotion', \
      'city', 'state', 'store_type', 'cluster', 'locale', 'locale_name', \
      'status', 'day', 'dayname', 'dayofyear', 'week', 'year', \
      'Terremoto', 'Futbol', 'CyberMonday', 'BlackFriday', 'Madre']].reset_index(drop=True)
clean.index.name='index'
clean

,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,locale,...,day,dayname,dayofyear,week,year,Terremoto,Futbol,CyberMonday,BlackFriday,Madre
index,,,,,,,,,,,,,,,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.000000,0,Quito,Pichincha,D,13,National,...,1,Tuesday,1,1,2013,False,False,False,False,False
1,2016-05-07,7,POULTRY,737.388977,0,Quito,Pichincha,D,8,National,...,5,Saturday,128,18,2016,False,False,False,False,False
2,2016-05-07,7,PREPARED FOODS,225.252991,0,Quito,Pichincha,D,8,National,...,5,Saturday,128,18,2016,False,False,False,False,False
3,2016-05-07,7,PRODUCE,5946.388184,0,Quito,Pichincha,D,8,National,...,5,Saturday,128,18,2016,False,False,False,False,False
4,2016-05-07,7,SCHOOL AND OFFICE SUPPLIES,0.000000,0,Quito,Pichincha,D,8,National,...,5,Saturday,128,18,2016,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.132996,0,Quito,Pichincha,B,6,NaN,...,1,Tuesday,227,33,2017,False,False,False,False,False
3000884,2017-08-15,9,PREPARED FOODS,154.552994,1,Quito,Pichincha,B,6,NaN,...,1,Tuesday,227,33,2017,False,False,False,False,False
3000885,2017-08-15,9,PRODUCE,2419.729004,148,Quito,Pichincha,B,6,NaN,...,1,Tuesday,227,33,2017,False,False,False,False,False


In [40]:
# save clean data as gzipped csv file
filename = path_data / 'clean_data.gz'
print(filename)
clean.to_csv(filename, compression=dict(method='gzip'))

input/clean_data.gz
